In [1]:
import saspy
sas = saspy.SASsession()

import pandas as pd
import numpy as np

Using SAS Config named: oda
SAS Connection established. Subprocess id is 14176



* splitting data into traing and testing.

In [2]:
%%SAS sas

libname l '/home/u49467254/CRM';


proc surveyselect
    data = l.loan_clean
    out = loan_ready1
    samprate = 0.70
    outall
    seed = 12345;
        samplingunit id;
run;

data l.loan_train l.loan_test;
set loan_ready1;
if selected = 1 then output l.loan_train;
else output l.loan_test;
run;

Selection Method,Simple Random Sampling
Sampling Unit Variable,id
Input Data Set,LOAN_CLEAN
Random Number Seed,12345
Sampling Rate,0.7
Sample Size,326400
Selection Probability,0.700001
Sampling Weight,1.428569
Output Data Set,LOAN_READY1


* coarse classification;

* character variables to be prepaired are 
'grade' 'home_ownership''verification_status' 'purpose' 'initial_list_status';

* continuous variables to be prepaired are
'term_int' 'emp_length_int' 'mths_since_issue_date'
'int_rate' 'funded_amnt'  'mths_since_earliest_cr_line'
'installment' 'delinq_2yrs' 'inq_last_6mths'
'open_acc' 'pub_rec' 'total_acc'
'acc_now_delinq';

* working with character values;

In [3]:
%%SAS sas

%let char= grade;
proc sql;
create table woe as 
select *,
 ((disc_bad - disc_good)*woe) as iv,
sum(abs(((disc_bad - disc_good)*woe))) as sum_IV from 
(
select *, ((good+bad)/sum((good+bad))) as prop_n,
         round((good/bad),1) as good_bad_odds,
        (good/sum(good)) as disc_good,
        (bad/sum(bad)) as disc_bad,
            log(((bad/sum(bad)))/((good/sum(good)))) as woe
from   (select &char., sum(bad in (0)) as good, sum(bad in (1)) as bad 
        from l.loan_train 
        group by &char.)
)
order by woe
;


proc sql;
select count(distinct(&char.)) as n_unique_&char. from l.loan_train;
quit;
ods graphics / reset width=6.4in height=4.8in imagemap;
proc sgplot data=woe;
    series x=&char. y=woe;
    scatter x=&char. y=woe /markerattrs=(symbol=circlefilled);
run;
ods graphics / reset;
proc print data=woe;
run;

/*
notes on 'grade':
"reference dummy" of 'grade A' for having low woe
it says as grade degrades from 'a' to 'f' odds of default increases
therefore g will represent higher chance of default
therefore we will keep grade a as ref dummy
*/

/*
notes on 'home_ownership':
attributes 'mortgage' and 'rent' together comprises 90% of data therefore will have 
separate dummies. rest 'any' 'other' 'none' will be composite dummy. 
odds ratio suggests 'own' should be taken with eigther 'mortgage' or 'rent' but since it has 
low woe value we will include it in the composite dummy.
"reference dummy" for home_ownership: 'home_ownership_any_own_other_none' for having low proportion.
"rest": 'mortgage' 'rent'
*/

/*
notes on 'verification_status':
all attributes have equal proportions
"reference": 'Source Verified' for having relatively low woe value.
"rest": 'Verified' 'Not Verified'
*/

/*
notes on the variable 'purpose':
there are 14 attributes.
"reference": attribute 'credit_card' has 20% prop and strong woe hence will be a separate dummy.
next dummy will comprise of: 'home_improvement' 'major_purchase' 'car' for having equal odds and car for having low prop.
next dummy will comprise of: 'debt_consolidation' for having 50% plus strong woe
next dummy will comprise of: 'wedding' 'house' 'vacation' - equal odds low prop.
next: 'renewable_energy' 'medical' 'other' 'moving'- graph shows moving is too close to renewable energy will low prop
finally: 'small_business' 'educational'
*/

/*
note on variable 'initial_list_status':
its a gender variable with weak IV.
*/

* working with continuous variables.

In [4]:
%%SAS sas

%let cont= acc_now_delinq;
proc sql;
select count(distinct(&cont.)) as unique from l.loan_train;
quit;



proc sql;
create table woe as 
select *,
 ((disc_bad - disc_good)*woe) as iv,
sum(abs(((disc_bad - disc_good)*woe))) as sum_IV from 
(
select *, ((good+bad)/sum((good+bad))) as prop_n,
         round((good/bad),1) as good_bad_odds,
         (good/sum(good)) as disc_good,
         (bad/sum(bad)) as disc_bad,
         log(((bad/sum(bad)))/((good/sum(good)))) as woe
from   (select &cont., sum(bad in (0)) as good, sum(bad in (1)) as bad 
        from l.loan_train 
        group by &cont.)
)
order by &cont.
;

proc sql;
select count(distinct(&cont.)) as n_unique_&cont. from l.loan_train;
quit;
ods graphics / reset width=6.4in height=4.8in imagemap;
proc sgplot data=woe;
    series x=&cont. y=woe;
    scatter x=&cont. y=woe /markerattrs=(symbol=circlefilled);
run;
ods graphics / reset;
proc print data=woe;
run;

/*
note on variable 'term_int':
has two unique and is already a dummy where 0 implies 36 months and 
 1 implies 60 months. hence reference is 36 months.
*/

/*
note on variable 'emp_length_int': has 10 unique values.
however IV is too low hence we will ignore this variable.
*/

/*
note on variable 'mths_since_issue_date': has 91 unique values.
note on variable 'int_rate': has 504 unique values.
note on variable 'funded_amnt': has 1347 unique values.
note on variable 'mths_since_earliest_cr_line': has 143 unique values. rest follows similarly:
'installment'- 51228
'delinq_2yrs'- 22 
'inq_last_6mths'- 26
'open_acc' - 60
 'pub_rec' - 24
 'total_acc' - 106
'acc_now_delinq' - 6
now, we will use the previous method for creating dummies if the values
are close to 20 unique values otherwise we will use hqbin procedure for analysis
however because of sas oda performance limitation we will proceed with the first 4
variables using proc hpbin making 20 bins as mentioned above  */

In [5]:
%%SAS sas

proc hpbin data=l.loan_train numbin=20;
     input mths_since_issue_date;
     input int_rate;
     input funded_amnt;
     input mths_since_earliest_cr_line_fix;
     ods output Mapping=Mapping;
run;

proc hpbin data=l.loan_train WOE BINS_META=Mapping;
     target bad/level=nominal order=desc;
run;

/*
based on the result we will reject 'funded amnt' and 'months since last credit line'
for having low IV score. now we will take 
*/

/*
notes on 'int_rate':
it tells that as int_rate increases probability to defult falls.
'6.452 <= int_rate < 7.484'- dummy1
'7.484 <= int_rate < 9.548'- dummy2
'9.548 <= int_rate < 12.644'- dummy3
'12.644 <= int_rate < 15.74' - dummy4
'15.74 <= int_rate < 20.9' - dummy5
'20.9 <= int_rate' - reference dummy6
*/
/*
notes on 'mths_since_issue_date':
'mths_since_issue_date < 40.5' - dummy1
'40.5 <= mths_since_issue_date < 49.5' - dummy2
'49.5 <= mths_since_issue_date < 58.5' - dummy3
'58.5 <= mths_since_issue_date' - ref dummy4 for having fluctuating woe (due to low proportions)
*/


**********dummy creation on both train and test********;

In [6]:
%%SAS sas


data l.train_ready ;
set l.loan_train ;


if purpose in ('credit_card') then purpose_new = 'purpose_credit_card'; 
if purpose in ('home_improvement' 'major_purchase' 'car') then purpose_new = 'purpose_homeimp_mjprc_car';
if purpose in ('debt_consolidation') then purpose_new = 'purpose_debt';
if purpose in ('wedding' 'house' 'vacation') then purpose_new = 'purpose_wed_house_vac';
if purpose in ('renewable_energy' 'medical' 'other' 'moving') then purpose_new = 'purpose_enrg_med_oth_mov';
if purpose in ('small_business' 'educational') then purpose_new = 'purpose_smbiss_edu';

if home_ownership in ('ANY' 'OTHER' 'NONE' 'OWN') then home_ownership_new = 'home_ownership_AONO';
if home_ownership in ('MORTGAGE') then home_ownership_new = 'home_ownership_mortgage';
if home_ownership in ('RENT') then home_ownership_new = 'home_ownership_rent';

if verification_status in ('Source Verified') then verification_status_new = 'verf_stat_Source_Verified';
if verification_status in ('Verified') then verification_status_new = 'verf_stat_Verified';
if verification_status in ('Not Verified') then verification_status_new = 'verf_stat_Not_Verified';


if int_rate LT 7.484 then int_rate_new = 'int_rate_lt_7.484';
if int_rate GE 7.484 and int_rate LT 9.548  then int_rate_new = 'int_rate_7.484-9.548';
if int_rate GE 9.548 and int_rate LT 12.644  then int_rate_new = 'int_rate_9.548-12.644';
if int_rate GE 12.644 and int_rate LT 15.74  then int_rate_new = 'int_rate_12.644-15.74';
if int_rate GE 15.74 and int_rate LT 20.9  then int_rate_new = 'int_rate_15.74-20.9';
if int_rate GE 20.9 then int_rate_new = 'int_GT_20.9';

if mths_since_issue_date LT 40.5 then msid_new = 'msid_LT_40.5';
if mths_since_issue_date GE 40.5 and mths_since_issue_date LT 49.5  then msid_new = 'msid_40.5-49.5';
if mths_since_issue_date GE 49.5 and mths_since_issue_date LT 58.5  then msid_new = 'msid_495-58.5';
if mths_since_issue_date GE 58.5 then msid_new = 'msid_GE_58.5';


run;

/*
ref dummy list:  
grade ref is 'A'  
home_ownership_AONO  
verf_stat_Source_Verified  
purpose_credit_card  
initial_list_status = 'f'  
simply use term_int. here 0 implies 36month is the reference.  
int_MT_20.9  
msid_GE_58.5  
*/

In [7]:
%%SAS sas

data l.test_ready ;
set l.loan_test ;


if purpose in ('credit_card') then purpose_new = 'purpose_credit_card'; 
if purpose in ('home_improvement' 'major_purchase' 'car') then purpose_new = 'purpose_homeimp_mjprc_car';
if purpose in ('debt_consolidation') then purpose_new = 'purpose_debt';
if purpose in ('wedding' 'house' 'vacation') then purpose_new = 'purpose_wed_house_vac';
if purpose in ('renewable_energy' 'medical' 'other' 'moving') then purpose_new = 'purpose_enrg_med_oth_mov';
if purpose in ('small_business' 'educational') then purpose_new = 'purpose_smbiss_edu';

if home_ownership in ('ANY' 'OTHER' 'NONE' 'OWN') then home_ownership_new = 'home_ownership_AONO';
if home_ownership in ('MORTGAGE') then home_ownership_new = 'home_ownership_mortgage';
if home_ownership in ('RENT') then home_ownership_new = 'home_ownership_rent';

if verification_status in ('Source Verified') then verification_status_new = 'verf_stat_Source_Verified';
if verification_status in ('Verified') then verification_status_new = 'verf_stat_Verified';
if verification_status in ('Not Verified') then verification_status_new = 'verf_stat_Not_Verified';


if int_rate LT 7.484 then int_rate_new = 'int_rate_lt_7.484';
if int_rate GE 7.484 and int_rate LT 9.548  then int_rate_new = 'int_rate_7.484-9.548';
if int_rate GE 9.548 and int_rate LT 12.644  then int_rate_new = 'int_rate_9.548-12.644';
if int_rate GE 12.644 and int_rate LT 15.74  then int_rate_new = 'int_rate_12.644-15.74';
if int_rate GE 15.74 and int_rate LT 20.9  then int_rate_new = 'int_rate_15.74-20.9';
if int_rate GE 20.9 then int_rate_new = 'int_GT_20.9';

if mths_since_issue_date LT 40.5 then msid_new = 'msid_LT_40.5';
if mths_since_issue_date GE 40.5 and mths_since_issue_date LT 49.5  then msid_new = 'msid_40.5-49.5';
if mths_since_issue_date GE 49.5 and mths_since_issue_date LT 58.5  then msid_new = 'msid_495-58.5';
if mths_since_issue_date GE 58.5 then msid_new = 'msid_GE_58.5';


run;